In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14407,2024-03-11,Romênia Divisão A,14:00,CSU Sibiu,Dinamo Bucharest,1.57,2.31,154.5,1.87,1.79,-4.5,1.98,2.24,6cRG4Qp0,0.636943,0.432900,0.534759,0.558659,0.069843,223.524,173.893348,0.777963,0.6,1.341641,2.236068,137.60,2.850,2.410705,0.845862,-14.0,185.192,86.452128,0.466824,0.6,1.341641,2.236068,312.66,2.494,0.989434,0.396726,-57.0,80,81,1.72,3.86,147.068,216.782,0.269721,0.030912,0.087132,1.03,0.206,2.766990,0.000000,0.0,0.000000,-3.63,-0.726,-1.804408,0.000000,0.0,0.000000
14408,2024-03-11,República Tcheca Nbl,15:00,Decin,Opava,2.01,1.73,166.5,1.80,1.86,1.5,1.86,1.60,lUW1LLU7,0.497512,0.578035,0.555556,0.537634,0.075547,189.690,134.275844,0.707870,1.2,1.643168,1.369306,102.03,2.384,1.458194,0.611659,-36.0,188.046,143.105000,0.761011,1.2,1.643168,1.369306,439.24,2.296,1.872279,0.815453,-4.0,57,79,1.79,5.56,104.844,216.596,0.105877,0.023184,0.106270,-1.48,-0.296,-3.412162,0.000000,0.0,0.000000,-1.11,-0.222,-3.288288,0.000000,0.0,0.000000
14409,2024-03-11,Lituânia Nkl,08:00,Zalgiris Kaunas 2,Kretinga,1.13,5.40,151.5,1.85,1.85,0.0,0.00,0.00,AsjmZ9id,0.884956,0.185185,0.540541,0.540541,0.070141,157.080,53.176657,0.338532,0.6,1.341641,2.236068,131.12,2.434,1.048632,0.430827,-24.0,200.468,55.464063,0.276673,2.4,1.341641,0.559017,180.00,2.564,1.058834,0.412962,25.0,88,72,1.49,2.50,142.772,260.712,0.924761,0.000000,NaN,-2.21,-0.442,-0.294118,0.660570,0.3,-0.360570,1.43,0.286,15.384615,0.351481,0.5,0.148519
14410,2024-03-11,Lituânia Lkl,13:50,Neptunas,Siauliai,1.45,2.61,168.5,1.80,1.86,0.0,0.00,0.00,dCNg2rOj,0.689655,0.383142,0.555556,0.537634,0.072797,214.448,122.166523,0.569679,1.2,1.643168,1.369306,197.34,2.718,1.773787,0.652607,-22.0,267.678,127.729316,0.477175,1.2,1.643168,1.369306,140.62,3.398,1.817036,0.534737,-34.0,78,89,2.53,1.58,198.908,282.202,0.404061,0.023184,NaN,-2.44,-0.488,-0.922131,0.000000,0.0,0.000000,0.68,0.136,11.838235,0.000000,0.0,0.000000
14411,2024-03-11,Libano Divisão 1,16:00,Antonine,Mayrouba,1.46,2.59,155.5,1.87,1.79,0.0,0.00,0.00,ABP5GrqN,0.684932,0.386100,0.534759,0.558659,0.071032,330.566,126.752360,0.383440,0.6,1.341641,2.236068,410.40,4.202,1.513446,0.360173,-63.0,342.586,252.959645,0.738383,0.0,0.000000,NaN,534.40,4.758,3.177935,0.667914,-81.0,80,80,5.13,6.68,280.472,326.512,0.394583,0.030912,NaN,2.01,0.402,1.144279,0.000000,0.0,0.000000,-5.00,-1.000,-1.590000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14471,2024-03-11,China Wcba Feminina,08:30,Liaoning F,Jiangsu Phoenix F,12.50,1.01,146.5,1.86,1.84,0.0,0.00,0.00,CfnwzMXL,0.080000,0.990099,0.537634,0.543478,0.070099,355.608,279.643231,0.786381,1.2,1.643168,1.369306,97.27,5.182,3.843269,0.741658,-43.0,149.720,121.871246,0.813994,1.2,1.643168,1.369306,99.06,2.160,2.111279,0.977444,10.0,71,78,1.37,1.27,204.750,159.174,1.202762,0.007644,NaN,-0.46,-0.092,-125.000000,0.451209,0.5,0.048791,-0.81,-0.162,-0.061728,0.598409,0.5,-0.098409
14472,2024-03-11,China Wcba Feminina,08:30,Hebei F,Xinjiang F,2.76,1.41,143.5,1.83,1.87,0.0,0.00,0.00,8QX8Uaeq,0.362319,0.709220,0.546448,0.534759,0.071539,119.650,71.073394,0.594011,2.4,1.34

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14407,14:00,Romênia Divisão A,CSU Sibiu,Dinamo Bucharest,1.57,Back Home
14408,15:00,República Tcheca Nbl,Decin,Opava,2.01,Back Home
14412,11:45,Libano Divisão 1,Hoops,Champville,2.76,Back Home
14419,15:00,Israel Superliga,Galil Elyon,Ironi Kiryat Ata,1.68,Back Home
14450,20:30,Eua Nba,Cleveland Cavaliers,Phoenix Suns,1.81,Back Home
14453,20:00,Brasil Nbb,Pinheiros,Unifacisa,2.13,Back Home
14469,15:15,Dinamarca Liga De Basquete,Svendborg,Horsens,2.44,Back Home
14472,08:30,China Wcba Feminina,Hebei F,Xinjiang F,2.76,Back Home
